# Discussion 7

### Due Friday Feb 21, 11:59:59PM




---

## Lecture Review

In [83]:
# import libraries
import pandas as pd
import numpy as np
import os
import requests
import bs4

# Lecture Review: HTTP Requests and HTML

### Requests

* HTTP requests can retrieve data using `GET` requests.
* The python function `resp = requests.get(url)` returns a response object:
    - `resp.text` contains the body of the response (e.g. HTML)
    - `resp.status_code` contains the status code of the response (e.g. 200, which means 'ok')
    - `resp.body` contains the entire response content.
    
### Parsing HTML

* An HTML page may be described as a tree (Document Object Model)
    - The nodes are HTML tags that define regions in the page (i.e. `<tag>...</tag>`).
    - Two nodes are connected by an edge if one HTML tag is nested in the other.
    - The sub-tree of below a given node (HTML tag) consists of the portion of the HTML page contained within that HTML tag.
* `BeautifulSoup` parses an HTML document into its DOM (tree structure).
    - `.children` attribute of a node iterates through the sub-trees of the DOM.
    - `.find(tag)` method selects the sub-tree of the DOM that contains the specified tag-type.
    - `.attr` accesses the attributes of a given tag (e.g. the hyperlink reference).
    - `.text` accesses the text between the start (`<tag>`) and end (`</tag>`) tags for a given node.

# Parsing Countries


In this question you need to scrape the website `http://example.webscraping.com/` and collect all the countries countries listed on the site (not just the first page). 

*Side note:* We saw in the lecture that there is a method `pd.read_html` that allows you to read HTML tables into a list of DataFrame objects. You can test it out but please **DO NOT** use it in your solution. The purpose of this problem is for you to practice scraping using simple tags first, before you move on to more difficult problems. 

### The Plan

To get this information of the website, you'll go through the following steps:
1. Write a function to *sucessfully* request the website content of a single page.
1. Write a function to request the content of all relevant pages.
1. Write a function to parse the HTML and extract the countries of an already-requested page.
1. Put it all together.

Remember, you should always make as few requests as possible. For example, when developing your parsing logic, you should work with a single successful request already on your computer -- don't request the content from the website repeatedly! Breaking your work up into function as specified below will help you do this.

**Step 1**: Check the website first, count the number of pages you need to go over, look at the URL of each page. Write a list of URLs that you need to request (find a pattern and use code; do not copy and paste).

In [4]:
url_list = []
for num in range(0, 26):
    url_list.append('http://example.webscraping.com/places/default/index/' + str(num))

In [5]:
url_list

['http://example.webscraping.com/places/default/index/0',
 'http://example.webscraping.com/places/default/index/1',
 'http://example.webscraping.com/places/default/index/2',
 'http://example.webscraping.com/places/default/index/3',
 'http://example.webscraping.com/places/default/index/4',
 'http://example.webscraping.com/places/default/index/5',
 'http://example.webscraping.com/places/default/index/6',
 'http://example.webscraping.com/places/default/index/7',
 'http://example.webscraping.com/places/default/index/8',
 'http://example.webscraping.com/places/default/index/9',
 'http://example.webscraping.com/places/default/index/10',
 'http://example.webscraping.com/places/default/index/11',
 'http://example.webscraping.com/places/default/index/12',
 'http://example.webscraping.com/places/default/index/13',
 'http://example.webscraping.com/places/default/index/14',
 'http://example.webscraping.com/places/default/index/15',
 'http://example.webscraping.com/places/default/index/16',
 'http:

**Step 2:** Once you have an idea how the site works, you can start sending requests to the pages and collect results. Loop through the `url_list`, requesting each page, and collect the HTTP responses in a list.

In [105]:
# for url in url_list:
    # txt = requests.get(url).text
    # print(url)

**Step 3:** What happens when you try execute the loop above? Why do you get an exception? You need to modify your code to both (1) handle these exceptions and (2) still collect all the data on the website(s).

To do this, create a function `request_until_successful` that takes in a `url` (and any optional keywords you might find useful) and the number of re-try requests the function will attempt `N` and returns a *successful* response object (or `None` if the request was still not successful after `N` attempts).

*Remark 1:* Your retry logic should effectively allow you to successfully request data from the site in the shortest time possible!

*Remark 2:* remember your status-codes!

In [74]:
def request_until_successful(url, N):
    if N==0:
        return None
    
    rq = requests.get(url)
    
    if rq.status_code != 200 and N>0:
        return request_until_successful(url, N-1)
    
    return rq


In [78]:
resp = request_until_successful('http://example.webscraping.com/', N=1)
resp

<Response [200]>

**Step 4:** Write a function `get_responses` that takes in `url_list` and returns the successful responses to each request made from the `url_list`.

In [120]:
def get_responses(url_list):
    rqs = []
    for url in url_list:
        rqs.append(request_until_successful(url, 5))
    return rqs

**Step 5:** Write a function `parse_page` that takes in http-request response object and returns the countries listed on the page. Write a function `parse_pages` that takes in a list of response objects and returns all the countries listed on all the pages.

*Remark:* Make sure to check the source page of the website in order to find appropriate tags for you to use during the scraping.


In [121]:
def parse_page(resp):
    soup = bs4.BeautifulSoup(resp.text, 'lxml')
    temp = soup.find_all('table')[0].find_all('a')# .text
    results = []
    for th in temp:
        results.append(th.text.strip())
    return results

In [122]:
txt = request_until_successful('http://example.webscraping.com/places/default/index/0', N=1).text

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
soup = bs4.BeautifulSoup(txt, 'lxml')
temp = soup.find_all('table')[0].find_all('a')# .text
for th in temp:
    print(th.text.strip())
# temp.find_all('a')[0].text

In [123]:
def parse_pages(resps):
    results = []
    for resp in resps:
        results.append(parse_page(resp))
    return results

**Step 6:** Write a function `get_countries` of zero variables that puts all of this together and returns the list of countries found on all the pages.

In [124]:
def get_countries():
    resps = get_responses(url_list)
    unflatten = parse_pages(resps)
    # print(unflatten)
    results = [item for sublist in unflatten for item in sublist]
    return results

In [125]:
results = get_countries()

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
len(results)

In [ ]:
results

---
# Practice Problems

**Turn in the first two functions from the steps above**